In [ ]:
#import everything I need
import requests
import pickle
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
class player: #class for players
    def __init__(self):
        self
class game: #each object - a game between two players
    def __init__(self):
        self
#functions for files    
def writefile(file, data):
    with open(file+'.pkl', 'wb') as output:
        #save = data
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
        
def readfile(file):
    with open(file+'.pkl', 'rb') as input:
        return(pickle.load(input))  

#add nans to create an even matrix    
def boolean_indexing(v, fillval=np.nan):
    lens = np.array([len(item) for item in v])
    mask = lens[:,None] > np.arange(lens.max())
    out = np.full(mask.shape,fillval)
    out[mask] = np.concatenate(v)
    return out    

In [ ]:
df = readfile('plays_pandas_ttligapro_men_last_version')
df = df.drop(['date1', 'date2', 'tournament_name', 'score_sets', 'score_points',\
        'name1', 'name2', 'login1', 'login2', 'fntrdate1', 'fntrdate2', 'day1', 'day2',\
             'year1', 'year2', 'month1', 'month2', 'fntrday1', 'fntrmonth1', 'fntryear1',\
             'fntrday2', 'fntrmonth2', 'fntryear2', 'ttligapro_liga', 'set_1_score', 'set_2_score',\
             'set_3_score', 'set_4_score', 'set_5_score'], axis=1)
df = df.drop(['fntrrating1', 'fntrrating2', 'rating1', 'rating2', 'ranking1', 'ranking2'], axis=1) #data from the future=)
#need to find current fntrratings and rankings, rankings' percentiles since the database is growing
writefile('history_dataset_numeric', df)

In [ ]:
df = readfile('history_dataset_numeric')

In [ ]:
columns = ['current_rating1', 'current_rating2', 'rating_delta_for1', 'won_sets', 'lost_sets',
       'ttligapro_liga_rating_min', 'ttligapro_liga_rating_max',
       'set_1_score_1', 'set_1_score_2', 'set_points_1_total', 'set_2_score_1',
       'set_2_score_2', 'set_points_2_total', 'set_3_score_1', 'set_3_score_2',
       'set_points_3_total', 'set_4_score_1', 'set_4_score_2',
       'set_points_4_total', 'set_5_score_1', 'set_5_score_2',
       'set_points_5_total', 'total_points', 'dsf']

In [ ]:
#inefficient (takes several minutes) because of the loops, but works
for c in columns:
    print(c)
    df[c+'_vector'] = np.empty((len(df), 0)).tolist()
    for j in df['id12'].unique():
        s = []
        s1 = []
        for i in df[df['id12']==j].sort_values(['year', 'month', 'day']).iterrows():
            s.append(i[1][c])
            s1.append(s.copy())
            df.set_value(i[0], c+'_vector', s1.copy())
    writefile('history_dataset_numeric_full', df)        